Signal 6 : Timeo Martin, Arno Braz, Julie Le Ral

### Partie 2 - Reconnaissance automatique de voyelle 

In [114]:
import numpy as np
import msicpe.san as san
from plotly import express as px
import scipy.signal as signal

signal_dict = np.load('../Sons/etude/signal_6.npz') 
# signal_dict est un dictionnaire → contenant les données contenues dans le fichier signal.npz

#### 2.1 Lecture des fichiers et affichages

In [115]:
s = signal_dict['s'] # extraction du signal utile
t = signal_dict['t'] # extraction du vecteur temps associé au signal

# Signal chargé en fonction du temps
trace_signal = px.line(x=t, y=s, labels={'x': 'Temps (t)', 'y': 's'}, title='Signal 6 (Temps)', width=900).show()

# Calculer la Transformée de Fourier du signal
tf = san.trans_fourier(s, t)
trace_spectre = px.line(x=tf[0], y=abs(tf[1]), labels={'x': 'frequences (nu)', 'y': 'S'}, width=900).show()

# Logarithme du spectre
trace_log10_spectre = px.line(x=tf[0], y=np.log10(abs(tf[1])), labels={'x': 'frequences (nu)', 'y': 'S'}, title='Spectre du Signal 6 (Fréquence)', width=900).show()

### Observations du signal i_1 :
T = (0.5-0.36)/15 = 0.00933s  
nu0 = 107.2Hz -> Correspond à l'observation graphique  

Le graphique est plus lisible en échelle logarithmique et il est donc plus adapté (notament pour les hautes fréquences) pour lire les harmoniques distantes de la fondamentale.


#### 2.2 Apprentissage - Caractérisation des voyelles

In [116]:
# Calcul de la norme d'un vecteur
def norme(vecteur):
    somme_x2 = 0
    for x in vecteur:
        somme_x2 += x**2
    return np.sqrt(somme_x2)

In [117]:
def find_H(signal):    
    signal_dict = np.load(signal) 
    s = signal_dict['s']
    t = signal_dict['t']

    nu, tf_signal = san.trans_fourier(s, t)
    spectre = abs(tf_signal)
    spectre_normal = spectre / max(spectre)

    fondamental = san.detect_fondamentale(spectre_normal, nu)
    frequence_h = [i * fondamental for i in range(1, 40)]   #a voir

    harmonique_freq, harmonique_amp = san.detect_pics(spectre, frequence_h, nu)
    H1 = 0
    for i in harmonique_amp:
        H1 += 1
        if harmonique_amp[0] != 0 and (i / max(harmonique_amp)) <= 0.1:
            return H1
        

In [118]:
# Calcul du vecteur contenant les valeurs d’amplitude du spectre aux fréquences d’intérêt
def harmonique(signal, H,) : 
    signal_dict = np.load(signal) 
    s = signal_dict['s']
    t = signal_dict['t']

    nu, tf_signal = san.trans_fourier(s, t)
    spectre = abs(tf_signal)
    spectre_normal = spectre / max(spectre)

    fondamental = san.detect_fondamentale(spectre_normal, nu)
    frequence_h = [i * fondamental for i in range(1, H+2)]

    harmonique_freq, harmonique_amp = san.detect_pics(spectre, frequence_h, nu)
    vecteur_amplitudes = list()
    for i in range(H+1) : 
        amplitude = harmonique_amp[i]
        amplitude = abs(amplitude) + 1e-12 # éviter log(0)
        vecteur_amplitudes.append(np.log10(amplitude))

    return vecteur_amplitudes


In [119]:
liste_voyelles = ('a', 'e', 'i', 'o', 'u')
ref = dict()

# Calcul du vecteur valeur moyenne
def valeur_moyenne(vecteurs):
    vect = list()
    for y in range(len(vecteurs[0])):
        somme = 0
        for x in range(len(vecteurs)):
            somme += vecteurs[x][y]
        vect.append(somme / len(vecteurs))
    return np.array(vect)

In [120]:
H_list=list()
for voyelle in ('a', 'e', 'i', 'o', 'u'):
    for n in range(1, 6):
        if find_H(f'../Sons/apprentissage/{voyelle}_{n}.npz')!=None:
            H_list.append(find_H(f'../Sons/apprentissage/{voyelle}_{n}.npz'))
H=int(np.max(H_list))
print(f"la valeur de H calculée est {H}")

la valeur de H calculée est 13


In [121]:
# Création du dictionnaire des vecteurs de valeur moyenne de référence
for voyelle in liste_voyelles:
    vecteurs_harmoniques = list()
    for n in range(1, 6):
        vecteurs_harmoniques.append(harmonique(f'../Sons/apprentissage/{voyelle}_{n}.npz', H))
    ref[voyelle] = valeur_moyenne(vecteurs_harmoniques)
    print(f"Vecteur des amplitudes du logarithme au indices des harmoniques considérées pour {voyelle} :", vecteurs_harmoniques)

Vecteur des amplitudes du logarithme au indices des harmoniques considérées pour a : [[np.float32(2.6437879), np.float32(2.5499532), np.float32(1.9889917), np.float32(2.02466), np.float32(1.8389488), np.float32(1.8991343), np.float32(2.2215483), np.float32(2.4568698), np.float32(2.1545043), np.float32(1.5579339), np.float32(2.0463305), np.float32(2.1623087), np.float32(1.7389097), np.float32(1.1947979)], [np.float32(2.7475643), np.float32(2.639196), np.float32(2.0772843), np.float32(2.3525145), np.float32(2.040964), np.float32(2.5095627), np.float32(2.4611423), np.float32(2.211502), np.float32(1.860627), np.float32(1.9526925), np.float32(2.303959), np.float32(1.7837402), np.float32(1.2102594), np.float32(0.94713795)], [np.float32(1.5836105), np.float32(1.5836105), np.float32(1.5836105), np.float32(1.5836105), np.float32(1.5836105), np.float32(1.5836105), np.float32(1.5836105), np.float32(1.5836105), np.float32(1.5836105), np.float32(1.5836105), np.float32(1.5836105), np.float32(1.58361

In [122]:
# Affichage des vecteurs de valeur moyenne utilisés pour référence

# print('Vecteurs de valeur moyenne par voyelle : ')
# for clef, vect_voyelle in ref.items():
    # print(f'{clef} :', vect_voyelle)
    # print()

moyenne_harmoniques = px.line(x=[i for i in range(1, H+2)], y=[ref['a'], ref['e'], ref['i'], ref['o'], ref['u']], 
                              labels={'x': 'harmoniques', 'value': 'Amplitude Moyenne (log)', 'variable': 'Légende'},
                              title='Moyenne des harmoniques pour chaque voyelle',
                              width=900)

for i, trace in enumerate(moyenne_harmoniques.data):
    trace.name = f"'{liste_voyelles[i]}'"

moyenne_harmoniques.show()


#### 2.3 Test de la méthode

In [123]:
# Identifie par méthode la distance la voyelle la plus similaire au signal d'entrée
def compare(vecteur):
    dico_distances = dict()
    distance_min = ('a', norme(np.abs(vecteur-ref['a'])))
    dico_distances['a'] = distance_min[1]
    for voyelle in liste_voyelles[1:]:
        distance =  norme(np.abs(vecteur-ref[voyelle]))
        dico_distances[voyelle] = distance
        if distance_min[1] > distance:
            distance_min = (voyelle, distance)
    return distance_min[0], dico_distances

In [124]:
# Validation des tests
for voyelle in liste_voyelles:
    for n in range(6, 8):
        vect_inconnu = valeur_moyenne([harmonique(f'../Sons/test/{voyelle}_{n}.npz', H)])
        resultat, distances_tests = compare(vect_inconnu)
        print(f"Le {voyelle}_{n} est identifié comme la voyelle : '{resultat}'")

Le a_6 est identifié comme la voyelle : 'a'
Le a_7 est identifié comme la voyelle : 'a'
Le e_6 est identifié comme la voyelle : 'e'
Le e_7 est identifié comme la voyelle : 'e'
Le i_6 est identifié comme la voyelle : 'i'
Le i_7 est identifié comme la voyelle : 'i'
Le o_6 est identifié comme la voyelle : 'o'
Le o_7 est identifié comme la voyelle : 'o'
Le u_6 est identifié comme la voyelle : 'u'
Le u_7 est identifié comme la voyelle : 'u'


##### 2.4 Application au cas d’étude    

In [125]:
# Identification du signal 6
vect_signal_6 = valeur_moyenne([harmonique('../Sons/etude/signal_6.npz', H)])
resultat, distances_signal6 = compare(vect_signal_6)
print(f"Le signal_6 est identifié comme étant la voyelle : '{resultat}'")

Le signal_6 est identifié comme étant la voyelle : 'o'


#### Conclusion

In [126]:
for clef, valeur in distances_signal6.items():
    print(clef, ':', valeur)

a : 4.871967
e : 3.0887425
i : 2.7894382
o : 0.8885725
u : 3.1164348


Parmi toutes les voyelles, c’est pour « o » que la distance au signal_6 est la plus petite ; le signal est donc classé comme un « o ».​ Ils se partagent donc des caractérisques relativement similaire (plus qu'avec les autres signaux)